# WEBSCRAPING DATA FOR DIAMOND MARKET ANALYSIS

- This is the code to scrape the diamond data from https://www.rarecarat.com
- The data is simply stored in a list and saved in a .pkl file for processing with the diamond_model.ipynb file.


In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
import pandas as pd
import pickle

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options




In [2]:

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
#this url correspond to the above website with the GIA and AGS selected as the ratings agency
diamond_url = 'https://www.rarecarat.com/diamonds/313d0abd-f149-4b10-9d28-08d74804c706'

In [19]:
chrome_options = Options()
chrome_options.add_argument("--headless") #this is so we avoid actually rendering the webpage

driver = webdriver.Chrome(chromedriver,options=chrome_options)
driver.get(diamond_url)

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
def get_data(soup):

    rows = soup.find_all(class_='visible-xs custom-favourite-diamonds-my-stuff rc-search-results-inner')
    results = []
    for row in rows:
        temp = []
        for j in range(9):
            temp.append(row.find_all(class_='mobile-options-li')[j].get_text().strip().split()[0])
        results.append(temp)
    return results
    

In [6]:

def fill_price(driver,price,prev_price):
    price = str(price)
    oldprice = str(prev_price)
    fill_price = driver.find_element_by_xpath(
    '//*[@id="main"]/div/div/div[2]/div/div/div/div[1]/div[2]/div/fieldset/div[1]/input')
    #cur_price = fill_price.getText()
    #print(cur_price)
    for _ in range(len(oldprice)):
        fill_price.send_keys(Keys.BACKSPACE)
    #time.sleep(3)
    fill_price.send_keys(price)
    fill_price.send_keys(Keys.ENTER)
    time.sleep(2)

Let's do some scrolling, price entering, and data gathering.

# MAKE SURE TO CHANGE THE PICKLE FILENAME WHEN/IF A RESTART IS NECESSARY

Because we are scraping from a single machine and a single source it can take quite a while to load all the webpages. 
We therefore save often and every time we shut down the computer and move out of range of internet access, and change the range for price searching and the .pkl file name so as not to write over previously scraped data.

In [20]:
all_the_results = []
prv_price = {'price':628}
for x in range(779,1000):#once we get to 200,000 got by 1000
    new_price = 100*x
    fill_price(driver,str(new_price),prv_price['price'])
    prv_price['price'] = new_price
    for i in range(7):
        #Scroll
        driver.execute_script(
        "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        all_the_results.extend(get_data(soup))
    #Wait for page to load
        time.sleep(5)
    #after we are done scrolling for that price point, change price point and repeat
    with open('diamond_list16.pkl', 'wb') as picklefile:
        pickle.dump(all_the_results, picklefile)
    print('Current Price: {}'.format(new_price) )

Current Price: 77900
Current Price: 78000
Current Price: 78100
Current Price: 78200
Current Price: 78300
Current Price: 78400
Current Price: 78500
Current Price: 78600
Current Price: 78700
Current Price: 78800
Current Price: 78900
Current Price: 79000
Current Price: 79100
Current Price: 79200
Current Price: 79300
Current Price: 79400
Current Price: 79500
Current Price: 79600
Current Price: 79700
Current Price: 79800
Current Price: 79900
Current Price: 80000
Current Price: 80100
Current Price: 80200
Current Price: 80300
Current Price: 80400
Current Price: 80500
Current Price: 80600
Current Price: 80700
Current Price: 80800
Current Price: 80900
Current Price: 81000
Current Price: 81100
Current Price: 81200
Current Price: 81300
Current Price: 81400
Current Price: 81500
Current Price: 81600
Current Price: 81700
Current Price: 81800
Current Price: 81900
Current Price: 82000
Current Price: 82100
Current Price: 82200
Current Price: 82300
Current Price: 82400
Current Price: 82500
Current Price

WebDriverException: Message: unknown error: session deleted because of page crash
from tab crashed
  (Session info: headless chrome=77.0.3865.90)


In [21]:
#to kill the driver when we are done
driver.quit()